In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pairprog.objectstore import * 
from pairprog.assistant import * 
from pairprog.assistant  import logger as as_logger

In [3]:
import logging
logging.basicConfig()
as_logger.setLevel(logging.DEBUG)

In [4]:
rc = ObjectStore.new(name='barker_minio', bucket='agent')

In [5]:
import typesense
ts = typesense.Client(
            {
                "api_key": "xyz",
                "nodes": [{"host": "barker", "port": "8108", "protocol": "http"}],
                "connection_timeout_seconds": 1,
            }
        )

tool = PPTools(ts, rc.sub('pptools'), Path('/Volumes/Cache/scratch'))

assis = Assistant(tool, cache=rc)

messages = [
    {"role": "user", "content": 'tell me a joke'}
]

def dump(o):
    import json 
    print( json.dumps(o.model_dump(), indent=2))
    
len(assis.tokenizer.encode(json.dumps(tool.specification())))

1311

In [6]:
for t in tool.specification():
    print(t['function']['name'])

execute_python
faster_and_cheaper
get_document
read_file
search_documents
shell
smarter
specification
store_document
system_message
web_search
wikipedia_page
wikipedia_search
write_file


In [7]:
def get_sessions():
    
    sessions = set(e.split('/')[0] for e in list(rc.sub('session').list()))
    return list(reversed(sorted(sessions)))

def get_last_src():

    last_session  = get_sessions()[0]
    src = rc.sub('session/'+last_session)
    return src

def dump_sessions():
    last_session  = get_sessions()[-3]
    src = rc.sub('session/'+last_session)
    for e in src['responses']:
        print(e)
        print('-----')

src = get_last_src()

for e in src['messages']:
    print(e)
    print('-----')


{'role': 'user', 'content': 'what time will it be in one and one half hours? I am in the Pacific Time Zone, and find the time yourself. '}
-----
{'role': 'assistant', 'content': ''}
-----
{'content': '', 'function_call': None, 'role': 'assistant', 'tool_calls': [{'index': 0, 'id': 'call_9UM4svioDXFK8oC3gngq1xiG', 'function': {'arguments': '{\n  "code": "from datetime import datetime, timedelta\\nfrom pytz import timezone\\n\\n# Set the timezone\\ntime_zone = timezone(\'US/Pacific\')\\n\\n# Get the current time in the specified timezone\\ncurrent_time = datetime.now(time_zone)\\n\\n# Add 1.5 hours to the current time\\nfuture_time = current_time + timedelta(hours=1.5)\\n\\nfuture_time.strftime(\'%Y-%m-%d %H:%M:%S\')"\n}', 'name': 'execute_python'}, 'type': 'function'}]}
-----
{'tool_call_id': 'call_9UM4svioDXFK8oC3gngq1xiG', 'role': 'tool', 'name': 'execute_python', 'content': "unsupported operand type(s) for +: 'NoneType' and 'str'"}
-----
{'role': 'assistant', 'content': ''}
-----
{'c

In [8]:
from pathlib import Path
from pairprog.taskmachine import TaskManager
tm = TaskManager(None, None, Path.cwd())
tm

In [9]:
print("from datetime import datetime, timedelta\nimport pytz\n\n# Get the current time in Pacific Time Zone\npacific = pytz.timezone('US/Pacific')\ncurrent_time = datetime.now(pacific)\n\n# Add one and a half hours to the current time\ntime_in_one_and_half_hours = current_time + timedelta(hours=1.5)\n\ntime_in_one_and_half_hours.strftime('%Y-%m-%d %H:%M:%S')")

from datetime import datetime, timedelta
import pytz

# Get the current time in Pacific Time Zone
pacific = pytz.timezone('US/Pacific')
current_time = datetime.now(pacific)

# Add one and a half hours to the current time
time_in_one_and_half_hours = current_time + timedelta(hours=1.5)

time_in_one_and_half_hours.strftime('%Y-%m-%d %H:%M:%S')


In [12]:
z = """from datetime import datetime, timedelta
import pytz

# Get the current time in Pacific Time Zone
pacific = pytz.timezone('US/Pacific')
current_time = datetime.now(pacific)

# Add one and a half hours to the current time
time_in_one_and_half_hours = current_time + timedelta(hours=1.5)

time_in_one_and_half_hours.strftime('%Y-%m-%d %H:%M:%S')"""
tool.execute_python (code=z)

'2023-11-29 17:15:12'

In [15]:
tool.run_tool('execute_python', {'code': z})

!!!!! <bound method PPTools.execute_python of <pairprog.tool.PPTools object at 0x10bc14e90>>
!!!!!! {'code': {'code': "from datetime import timedelta, datetime\nfrom pytz import timezone\n\n# Get current time\npacific = timezone('US/Pacific')\nnow = datetime.now(pacific)\n\n# Add one and a half hours to current time\nfuture_time = now + timedelta(hours=1.5)\nfuture_time.strftime('%Y-%m-%d %H:%M:%S')"}}
!!!!! code {'code': "from datetime import timedelta, datetime\nfrom pytz import timezone\n\n# Get current time\npacific = timezone('US/Pacific')\nnow = datetime.now(pacific)\n\n# Add one and a half hours to current time\nfuture_time = now + timedelta(hours=1.5)\nfuture_time.strftime('%Y-%m-%d %H:%M:%S')"} must be a string


'code {\'code\': "from datetime import timedelta, datetime\\nfrom pytz import timezone\\n\\n# Get current time\\npacific = timezone(\'US/Pacific\')\\nnow = datetime.now(pacific)\\n\\n# Add one and a half hours to current time\\nfuture_time = now + timedelta(hours=1.5)\\nfuture_time.strftime(\'%Y-%m-%d %H:%M:%S\')"} must be a string'

In [17]:
t = {'code': "from datetime import timedelta, datetime\nfrom pytz import timezone\n\n# Get current time\npacific = timezone('US/Pacific')\nnow = datetime.now(pacific)\n\n# Add one and a half hours to current time\nfuture_time = now + timedelta(hours=1.5)\nfuture_time.strftime('%Y-%m-%d %H:%M:%S')"}
tool.execute_python(**t)

'2023-11-29 17:21:05'

In [5]:
from pairprog.tool import * 

def m( line, task_state):
    match line, task_state:
        case (None,''),TaskState.NONE: # No line, not in task
            return "A"
        case _,TaskState.NONE: # Line, in task
            return "A2"
        case _, TaskState.ANALYZE: 
            return "B"
        case _, TaskState.INTASK:
            return "C"
        case _, TaskState.AUTO_CONTINUE:
            return "D"
        case _,_:
            return "DEFAULT"


m('lello', TaskState.INTASK)

'C'